# Validation Plots
Load outputs from validate_physics.py and plot Flux PDF, Power Spectrum, Curvature.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

npz_path = 'validation_outputs.npz'
out_pdf = 'validation_report.pdf'

data = np.load(npz_path)
x_real = data['x_real']
x_gen = data['x_gen']
k = data['k']
pk_real = data['pk_real']
pk_gen = data['pk_gen']
curv_real = data['curv_real']
curv_gen = data['curv_gen']
wave = data['wave']
bound_arr = data['bound']
bound_min = float(bound_arr.min())
bound_max = float(bound_arr.max())
frac_real = float(data['frac_real'][0])
frac_gen = float(data['frac_gen'][0])

all_vals = np.concatenate([x_real.ravel(), x_gen.ravel()])
vmin, vmax = np.percentile(all_vals, [0.5, 99.5])
bins = np.linspace(vmin, vmax, 80)

curv_all = np.concatenate([curv_real.ravel(), curv_gen.ravel()])
cmin, cmax = np.percentile(curv_all, [0.5, 99.5])
cbins = np.linspace(cmin, cmax, 80)

fig, axes = plt.subplots(1, 3, figsize=(15, 4.8))

axes[0].hist(x_real.ravel(), bins=bins, density=True, alpha=0.6, label='Real')
axes[0].hist(x_gen.ravel(), bins=bins, density=True, alpha=0.6, label='Gen')
axes[0].set_title('Flux PDF')
axes[0].set_xlabel('Flux')
axes[0].set_ylabel('Probability Density')
axes[0].legend()

axes[1].plot(k, pk_real, label='Real')
axes[1].plot(k, pk_gen, label='Gen')
axes[1].set_title('Power Spectrum')
axes[1].set_xlabel('k (pixel^-1)')
axes[1].set_ylabel('P(k)')
axes[1].set_yscale('log')
axes[1].legend()

axes[2].hist(curv_real.ravel(), bins=cbins, density=True, alpha=0.6, label='Real')
axes[2].hist(curv_gen.ravel(), bins=cbins, density=True, alpha=0.6, label='Gen')
axes[2].axvline(bound_min, color='k', linestyle='--', linewidth=1.0, label='Lower bound (min/max)')
axes[2].axvline(bound_max, color='k', linestyle='--', linewidth=1.0)
axes[2].set_title('Curvature D = d2/dx2 log(F)')
axes[2].set_xlabel('D')
axes[2].set_ylabel('Probability Density')
axes[2].legend()
axes[2].text(
    0.02,
    0.98,
    f'Violations (D < bound):\nReal {frac_real:.2f}% | Gen {frac_gen:.2f}%',
    transform=axes[2].transAxes,
    va='top',
    ha='left',
    fontsize=9,
    bbox=dict(boxstyle='round', facecolor='white', alpha=0.8, edgecolor='0.7'),
)

fig.suptitle(f'Validation Report | lower bound range [{bound_min:.3e}, {bound_max:.3e}]', fontsize=12)
fig.tight_layout()
fig.savefig(out_pdf)
print(f'[INFO] Saved report: {out_pdf}')


In [ ]:
# Plot bound curve vs sample curvatures
sample_idx = 0
valid = slice(2, -2)

plt.figure(figsize=(10, 6))
plt.plot(wave[valid], bound_arr[valid], 'r--', lw=2.5, label='Theoretical Bound (-1/sigma^2)')
plt.plot(wave[valid], curv_gen[sample_idx, valid], 'k-', alpha=0.5, lw=0.7, label='Generated Curvature')
plt.plot(wave[valid], curv_real[sample_idx, valid], 'b-', alpha=0.5, lw=0.7, label='Real Curvature')
plt.ylim(-1.0, 0.1)
plt.xlabel('Wavelength [A]')
plt.ylabel('Curvature d2 log F')
plt.title('Curvature Constraint Check (Linear Grid)')
plt.legend(loc='lower right')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig('curvature_check_linear.png')
print('[INFO] Saved plot: curvature_check_linear.png')
